In [29]:
import os
import re
import streamlit as st
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import Ollama

# === CONFIGURATION ===
index_folder = r"F:\Prashant Project\Python AI ML Projects\ClassifiedDocsVectorIndex"

# === LOAD EMBEDDINGS & VECTORSTORE ===
embeddings = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={"device": "cpu"}
)

vectorstore = FAISS.load_local(index_folder, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# === LLM & QA CHAIN ===
llm = Ollama(model="llama3")
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

# === STREAMLIT UI ===
st.set_page_config(page_title="PDF QA", layout="centered")
st.title("🕵️‍♂️ Intelligence Q&A")

question = st.text_input("Ask a question:", "")

if st.button("Get Answer") and question.strip():
    with st.spinner("Thinking..."):
        result = qa_chain(question)

        st.markdown("### 📄 Answer")
        st.write(result['result'])

        relevant_docs_with_scores = vectorstore.similarity_search_with_score(question, k=5)

        st.markdown("### 📚 Source Snippets")
        for i, (doc, score) in enumerate(relevant_docs_with_scores):
            filename = os.path.basename(doc.metadata.get("source", "Unknown"))
            snippet = doc.page_content[:1000]

            # Highlight matched answer
            answer_text = result['result']
            escaped_answer = re.escape(answer_text[:100])
            snippet = re.sub(f"({escaped_answer})", r'<mark style="background-color: yellow">\1</mark>', snippet, flags=re.IGNORECASE)

            with st.expander(f"{i+1}. {filename} (Similarity: {score:.2f})"):
                st.markdown(snippet + "...", unsafe_allow_html=True)


C:\Users\prashanti\.conda\envs\rag-env\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2025-06-03 16:35:55.027 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 16:35:55.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 16:35:55.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 16:35:55.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 16:35:55.049 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-03 16:35:55.050 Thread 'MainThread': missing ScriptRunContext!